<a href="https://colab.research.google.com/github/kavsraj/Applied_Datascience_Capstone_Project/blob/master/Segmenting_%26_Clustering_Neighborhoods_in_Toronto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###**Segmenting & Clustering Neighborhoods in Toronto - 2**

#####Setting up the environment


In [1]:
import pandas as pd
!pip -q install lxml

!pip install requests
import requests 
import io 

import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


##### Scraping the webpage to obtain the data and create the Dataframe



In [2]:
df=pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695")[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#####Cleaning Data and arrive at desired output

In [3]:
# Ignore cells with a borough that is 'Not assigned'.
df = df[df.Borough != 'Not assigned']

# Neighborhoods separated with a comma.
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x:', '.join(x)).to_frame().reset_index()

# Neighborhood same as the borough. 
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head(15)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#####get the latitude & longitude coordinates of each neighborhood & display the Dataframe

In [0]:
url1="http://cocl.us/Geospatial_data"
req=requests.get(url1).content
dec=pd.read_csv(io.StringIO(req.decode('utf-8')))dec.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(dec, df, on='Postcode')

df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df